# Creating E-Books (.epub) in python using ebooklib


This post includes code and notes from [ebooklib](https://github.com/aerkalov/ebooklib/) and [make-an-ebook](http://rtpg.co/2016/09/17/make-an-ebook.html).

In [1]:
import os
import requests

from ebooklib import epub
from pyquery import PyQuery as pq

In [2]:
# coding=utf-8

url = "http://example.com/%s.html"
build_dir = "build/"

if not os.path.exists(build_dir):
    os.makedirs(build_dir)
    
source_urls = [url % i for i in range(1,2)]

urls = [
    (build_dir + "%s.html" % i, url % i) for i in range(1,2)
]

for filename, url in urls:
    print("Getting ", url)
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)

Getting  http://example.com/1.html


In [6]:
if __name__ == '__main__':
    book = epub.EpubBook()

    # add metadata
    book.set_identifier('sample123456')
    book.set_title('Sample book')
    book.set_language('en')

    book.add_author('Example Author')

    # intro chapter
    c1 = epub.EpubHtml(title='Introduction', file_name='intro.xhtml', lang='en')
    c1.content=u'<html><head></head><body><h1>Introduction</h1><p>Introduction paragraph here.</p></body></html>'

    # about chapter
    c2 = epub.EpubHtml(title='About this book', file_name='about.xhtml')
    c2.content='<h1>About this book</h1><p>Text about his book.</p>'

    # add chapters to the book
    book.add_item(c1)
    book.add_item(c2)
    
    # create table of contents
    # - add section
    # - add auto created links to chapters

    book.toc = (epub.Link('intro.xhtml', 'Introduction', 'intro'),
                 (epub.Section('Languages'),
                 (c1, c2))
                )

    # add navigation files
    book.add_item(epub.EpubNcx())
    book.add_item(epub.EpubNav())

    # define css style
    style = '''
@namespace epub "http://www.idpf.org/2007/ops";
body {
    font-family: Cambria, Liberation Serif, Bitstream Vera Serif, Georgia, Times, Times New Roman, serif;
}
h2 {
     text-align: left;
     text-transform: uppercase;
     font-weight: 200;     
}
ol {
        list-style-type: none;
}
ol > li:first-child {
        margin-top: 0.3em;
}
nav[epub|type~='toc'] > ol > li > ol  {
    list-style-type:square;
}
nav[epub|type~='toc'] > ol > li > ol > li {
        margin-top: 0.3em;
}
'''

    # add css file
    nav_css = epub.EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)
    book.add_item(nav_css)

    # create spine
    book.spine = ['nav', c1, c2]

    # create epub file
    epub.write_epub('test.epub', book, {})